# CIS600 - Social Media & Data Mining
###  
<img src="https://www.syracuse.edu/wp-content/themes/g6-carbon/img/syracuse-university-seal.svg?ver=6.3.9" style="width: 200px;"/>

# Clustering & Mixture Models
#### *Taken mostly from Bishop's text on machine learning*
###  April 16, 2018

# $k$-means

## Optimization Formulation

> Everything is an optimization problem, yes. Realize that, and then get over it as soon as you can. (Boyd)

### We have used $k$-means and discussed the algorithm. Here is the setup:

### We have a dataset $\{x_1,\ldots,x_N\}$ of observations from $\mathbb{R}^D$. We seek clusters with centers $\mu_k \in \mathbb{R}^D$. *The cluster centers are the things we are choosing*. We are choosing them so that the sum of squared distances are minimized. That is, we are minimizing:

## $J = \sum_{n=1}^N\sum_{k=1}^K r_{nk} \|x_n - \mu_k\|^2$

### where

## $ r_{nk} = \begin{cases} 1 & x_n \in \text{Cluster } k \\
                             0 & \text{otherwise}   \end{cases}$

### The standard algorithm can be broken down into two steps

### *Expectation* - minimize $J$ wrt the $r_{nk}$, leaving $\mu_k$ fixed.

### *Maximization* - minimize $J$ wrt the $\mu_k$ leaving $r$ fixed.

### (This is an instance of a more general method called EM.)

### The expectation step makes the following update to the $r$ values:

## $ r_{nk} = \begin{cases} 1 & \text{ if } k= argmin_j \|x_n - \mu_j\|^2 \\
                             0 & \text{otherwise}   \end{cases}$

### The maximization step is done by calculus. The partial equations

## $2\sum_{n=1}^Nr_{nk}(x_n - \mu_k) = 0$

### lead to the formula for $\mu_k$

## $ \mu_k = \frac{\sum_nr_{nk}x_n}{\sum_nr_{nk}}$

### Which is exactly the mean of all the things that had been assigned to cluster $k$ in the previous step.

### We keep doing this until the clusters stop changing (or until a certain number of iterations has been completed.

### Let's get some data. This is a good opportunity to use what we already know. Bishop used the famous *Old Faithful* dataset. It can be found, among other places, [here](http://www.stat.cmu.edu/~larry/all-of-statistics/=data/faithful.dat). Let's pull it into our python session.

In [1]:
import numpy as np
import urllib.request
url = 'http://www.stat.cmu.edu/~larry/all-of-statistics/=data/faithful.dat'
with urllib.request.urlopen(url) as response:
    olf = response.readlines()

### Convert to strings

In [3]:
olf = [x.decode('utf-8') for x in olf]

### Strip off endline garbage

In [4]:
olf = [x.strip() for x in olf]

In [5]:
for x in olf: print(x)

Old Faithful Geyser Data

Description: (From R manual):

Waiting time between eruptions and the duration of the eruption
for the Old Faithful geyser in Yellowstone National Park, Wyoming,
USA.

A data frame with 272 observations on 2 variables.

eruptions  numeric  Eruption time in mins
waiting    numeric  Waiting time to next eruption

References:

Hardle, W. (1991) Smoothing Techniques with Implementation in S.
New York: Springer.

Azzalini, A. and Bowman, A. W. (1990). A look at some data on the
Old Faithful geyser. Applied Statistics 39, 357-365.





eruptions waiting
1       3.600      79
2       1.800      54
3       3.333      74
4       2.283      62
5       4.533      85
6       2.883      55
7       4.700      88
8       3.600      85
9       1.950      51
10      4.350      85
11      1.833      54
12      3.917      84
13      4.200      78
14      1.750      47
15      4.700      83
16      2.167      52
17      1.750      62
18      4.800      84
19      1.600      52
20

### Where does the data start?

In [6]:
olf.index('eruptions waiting')

25

### Where does it end?

In [7]:
olf[-1]

'272     4.467      74'

### Let's put it in an array.

In [5]:
dat = [[float(y) for y in x.split()[1:]] for x in olf[26:]]
olf_mat = np.array(dat)

In [6]:
olf_mat.shape

(272, 2)

### Checks out. Let's apply $k$-means to this data. We'll take $k = 2$.

### First, let's initialize with random centers from our *Old Faithful* datapoints.

In [7]:
# Choosing Centers
#mu = olf_mat[[3, 5]]

mu = np.array([[2,80],[4, 50]])


# Creating r_nk
ar = np.zeros((len(olf_mat),2))

# Initializing r_nk
for row in range(olf_mat.shape[0]):
    dists = np.linalg.norm(row - mu, axis=1)
    smallest = min(dists)
    center = list(dists).index(smallest)
    ar[row,center] = 1

### Let's have functions to execute the two steps.

In [8]:
# This does the expectation step
def estep():
    # Make sure we're dealing with the right one
    global ar
    # Reset the matrix
    ar = np.zeros((len(olf_mat),2))
    for row in range(olf_mat.shape[0]):
        dists = np.linalg.norm(olf_mat[row] - mu, axis=1)
        smallest = min(dists)
        center = list(dists).index(smallest)
        ar[row,center] = 1
        
# This does the maximization step
def mstep():
    for row in range(len(mu)):
        in_cluster = olf_mat[np.where(ar[:,row])]
        mu[row] = np.sum(in_cluster,axis=0) / in_cluster.shape[0]

### Let's now plot the clusters we are computing

In [11]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

mstep()
estep()


# Build a figure
p = figure(title="Old Faithful", toolbar_location=None)
p.grid.grid_line_color = None
p.background_fill_color = "#eeeeee"

def mscatter(p, x, y, marker, fill):
    p.scatter(x, y, marker=marker, size=15,
              line_color="navy", fill_color=fill, alpha=0.5)


# First cluster
C1 = olf_mat[np.where(ar[:,0])]

# Second Cluster
C2 = olf_mat[np.where(ar[:,1])]
    
# Throw in clusters
mscatter(p, C1[:,0], C1[:,1], "circle", "blue")
mscatter(p, C2[:,0], C2[:,1], "square", "orange")



show(p)

Loading BokehJS ...

## Checking Progress - plots of $J$

### Recall our cost function. Let's make it into a python function. What should it take as input?

In [77]:
ar[4,1]

0.0

In [9]:
def Jay(ars,m):
    total = 0
    for row in range(olf_mat.shape[0]):
        for k in range(2):
            if ar[row,k]:
                total += np.linalg.norm(olf_mat[row] - mu[k])**2
    return total

In [80]:
Jay(ar,mu)

8988.134974999995

### Let's reinitialize, and then make a plot.

In [12]:
# Choosing Centers
#mu = olf_mat[[3, 5]]

mu = np.array([[2,80],[4, 50]])


# Creating r_nk
ar = np.zeros((len(olf_mat),2))

# Initializing r_nk
for row in range(olf_mat.shape[0]):
    dists = np.linalg.norm(row - mu, axis=1)
    smallest = min(dists)
    center = list(dists).index(smallest)
    ar[row,center] = 1

In [13]:
# Build a figure
f = figure(title="Cost Function J", toolbar_location=None)
f.grid.grid_line_color = None
f.background_fill_color = "#eeeeee"

# X and Y coords
X, Y = range(4),[]

for i in X:
    mstep()
    estep()
    Y.append(Jay(ar,mu))

# A line separating them
f.line(X,Y, line_color='green')


show(f)

### Pretty much what you'd expect.

## Elbow Method - plots of $SSE$ to work out $k$

### A related but different plot can help us to decide which $k$ to use. Let's generate some more complex data, which we'll also use in the final example.

In [23]:
C1[:,0]

array([-5.10527777, -0.2006228 ])

In [25]:
mn = [-6,0]
cov = [[1,0],[0,.2]]

mn1 = [1,1]
cov1 = [[3,-2],[-2,3]]

mn2 = [6,2]
cov2 = [[.2,0],[0,1]]

f1 = figure(title="Fake Data", toolbar_location=None)
f1.grid.grid_line_color = None
f1.background_fill_color = "#eeeeee"



C1 = np.random.multivariate_normal(mn, cov, 500).T
mscatter(f1, C1[0,:], C1[1,:], "circle", "blue")

C2 = np.random.multivariate_normal(mn1, cov1, 500).T
mscatter(f1, C2[0,:], C2[1,:], "circle", "red")

C3 = np.random.multivariate_normal(mn2, cov2, 500).T
mscatter(f1, C3[0,:], C3[1,:], "circle", "yellow")

show(f1)

### Concatenating...

In [33]:
X = np.concatenate((C1,C2,C3),axis=1)

### Need to transpose the shape...

In [36]:
X = X.transpose()

### Finally, let's plot

In [37]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

f3 = figure(title="Elbow Method", toolbar_location=None)
f3.grid.grid_line_color = None
f3.background_fill_color = "#eeeeee"

# 
f3.line(K,distortions, line_color='green')


show(f3)

### What else can you do? See the [Silhouette Method](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html).

# Gaussian Mixture Models

### Now let's look at some statistical analysis that happens to lend itself to clustering.

### A *Gaussian mixture distribution* takes the form

## $p(x) = \sum_{k=1}^K \pi_k\mathcal{N}(x \mid \mu_k,\Sigma_k)$

### We define a $K-$dimensional random variable $\bf z$ such that $z_k \in \{0,1\}$ and $\sum_k z_k = 1$.

### The joint distribution for $x$ (data) and this variable is expressed in the form

## $p(x,z) = p(z)p(x \mid z)$

### Algebra gives the quantity we want (the one that helps with classification):

## $p(z_k = 1 \mid x) = \frac{\pi_k\mathcal{N}(x \mid \mu_k,\Sigma_k)}{\sum_j^K \pi_j\mathcal{N}(x \mid \mu_k,\Sigma_k)}$

### *This expresses the probability of belonging to cluster $k$*. It is referred to as the *responsibility* that the $k^{th}$ Gaussian distribution takes for explaining the observations $x$.  

### The *log-likelihood* is the function to be optimized, and the decision variables are the parameters for the distribution, i.e. $\pi$, $\mu$ and $\Sigma$.

## $ \ln p(x \mid \pi, \mu, \Sigma)$

### It can be optimized via an *Expectation-Maximization* algorithm:

### EM for Gaussian Mixtures (Bishop)

1. Initialize $\mu_k$, $\Sigma_k$ and $\pi_k$
2. (E-step) Evaluate *responsibilities* using *current parameter estimates*.
3. (M-step) Re-estimate parameters using current responsibilities. Assignments in this step are derived from Langrange multipliers (gives conditions for optimality).
4. Evaluate the log-likelihood and check for its convergence or else that of the parameters. If not converged, start again with Step 2.

### Example

In [38]:
from sklearn import mixture
gmm = mixture.GaussianMixture(n_components=3, covariance_type='full',
                              max_iter=100).fit(X)

In [42]:
predictions = gmm.predict(X)

### What clusters have we found?

In [47]:
D1 = X[np.where(predictions==0)].transpose()
D2 = X[np.where(predictions==1)].transpose()
D3 = X[np.where(predictions==2)].transpose()

In [48]:
f4 = figure(title="Found Clusters", toolbar_location=None)
f4.grid.grid_line_color = None
f4.background_fill_color = "#eeeeee"



mscatter(f4, D1[0,:], D1[1,:], "circle", "blue")

mscatter(f4, D2[0,:], D2[1,:], "circle", "red")

mscatter(f4, D3[0,:], D3[1,:], "circle", "yellow")

show(f4)

### Well, we got the colors flipped, but those look about the same as the sets generated!